# MCVTS ¯\\_(ツ)_/¯ League History
Established in 2013, this fantasy football league has a storied history spanning 18+ managers and 2 fantasy football platforms.

In [ ]:
# Don't mind me, just importing data.
from datetime import datetime

import pandas as pd
from tabulate import tabulate

managers = pd.read_csv("data/managers.csv")
players = pd.read_csv("data/players.csv")

finish = pd.read_csv("data/finish.csv")
finish = finish.merge(managers, on="manager_id")

games = pd.read_csv("data/games.csv")
games = games.merge(managers, on="manager_id")

playoff_start_weeks = []  # list of (season, first week of playoffs)
for season in range(2013, 2021):
    playoff_start_weeks.append((season, 15))
for season in range(2021, datetime.now().year + 1):  # 17 week NFL seasons smh
    playoff_start_weeks.append((season, 16))

regular_season_games = games.copy(deep=True)
for season, start_week in playoff_start_weeks:
    regular_season_games = regular_season_games.drop(
        regular_season_games[
            (regular_season_games['season_id'] == season) &
            (regular_season_games['week_id'] >= start_week)
        ].index
    )

# The same game stats may be present multiple times if the opponent had co-managers, so ensure each
# manager is only represented once here.
regular_season_games_deduped = regular_season_games.drop_duplicates(
    subset=['manager_id', 'season_id', 'week_id'],
    keep='first',
)

lineups = pd.read_csv("data/lineups.csv")
lineups = lineups.merge(managers, on="manager_id")
lineups = lineups.merge(players, on="player_id")

seasons = pd.read_csv("data/seasons.csv")
seasons = seasons.merge(managers, on="manager_id")
seasons['total_games'] = seasons[['wins', 'ties', 'losses']].sum(axis=1)

## League Members
Here are all the members in the league, past and present.

In [ ]:
seasons_count = seasons.groupby(['manager_id', 'manager_name'])["manager_id"].count() \
    .reset_index(name="num_seasons") \
    .sort_values(by=["num_seasons", "manager_name"], ascending=[False, True]) \
    [["manager_name", "num_seasons"]]

print(tabulate(seasons_count, headers='keys', tablefmt='psql', showindex=False))

# League champions
Here, there be champions (and runner-ups).

In [ ]:
champion_results = finish.loc[(finish['final_standing'] == 1) | (finish["final_standing"] == 2)]
champion_results = champion_results.sort_values(by=["season_id", "final_standing"], ascending=[False, True])

previous_season_id = None
last_rank = None
output = ""
for _, row in champion_results.iterrows():
    if row["season_id"] != previous_season_id:
        if output:
            print(output + ")")

        output = f"{row['season_id']} -"

    current_rank = row["final_standing"]
    if current_rank == 1:
        if last_rank == current_rank:
            output += ","
        
        output += f" {row['manager_name']}"
    else:
        if last_rank == 1:
            output += f" ({row['manager_name']}"
        elif last_rank == 2:
            output += f", {row['manager_name']}"

    previous_season_id = row["season_id"]
    last_rank = current_rank

print(output + ")")

# Most Championships
The winningest winners!

In [ ]:
champions = finish.loc[(finish['final_standing'] == 1)]
champions = champions.groupby(['manager_id', 'manager_name'])["manager_id"].count() \
    .reset_index(name="count").sort_values(by=["count", "manager_name"], ascending=[False, True]) \
    [["manager_name", "count"]]

print(tabulate(champions, headers='keys', tablefmt='psql', showindex=False))

# Most Regular Seasons Wins
Unfortunately, you don't get a trophy for this.

In [ ]:
wins_count = seasons.groupby(['manager_id', 'manager_name'])['wins'].sum() \
    .reset_index(name="total_wins").sort_values(
        by=['total_wins', 'manager_name'],
        ascending=[False, True]
    )[["manager_name", "total_wins"]]

print(tabulate(wins_count, headers='keys', tablefmt='psql', showindex=False))

# Highest Regular Season Winning Percentage
Some say wins are a longevity stat.

In [ ]:
total_wins = seasons.groupby(['manager_id', 'manager_name'])['wins'].sum() \
    .reset_index(name='total_wins').sort_values(by=['manager_id'], ascending=True)
    
total_games = seasons.groupby(['manager_id', 'manager_name'])['total_games'].sum() \
    .reset_index(name='total_games').sort_values(by=['manager_id'], ascending=True)

win_percentage = total_wins[['manager_id', 'manager_name']]
win_percentage['win_percentage'] = total_wins['total_wins'] / total_games['total_games']
win_percentage = win_percentage.sort_values(
        by=['win_percentage', 'manager_name'], 
        ascending=[False, True]
    )[["manager_name", "win_percentage"]]

print(tabulate(win_percentage, headers='keys', tablefmt='psql', showindex=False))

# Total Points Scored (Regular Season)
The 2013 Denver Broncos scored the most points in the regular season but lost 43-8 in the Super Bowl.

In [ ]:
total_points_scored_from_seasons = \
    seasons.groupby(['manager_id', 'manager_name'])['points_for'].sum() \
        .reset_index(name='total_points').sort_values(
            by=['total_points', 'manager_name'], ascending=[False, True]
        )[["manager_name", "total_points"]]

print(tabulate(total_points_scored_from_seasons, headers='keys', tablefmt='psql', showindex=False))

# Average Points Per Game (Regular Season)
Against these managers, you're gonna have a bad time.

In [ ]:
total_points_scored_from_games = \
    regular_season_games_deduped.groupby(['manager_id', 'manager_name'])['points_for'].sum() \
        .reset_index(name='total_points').sort_values(by='manager_id', ascending=True)

total_games_from_games = \
    regular_season_games_deduped.groupby(['manager_id', 'manager_name'])['manager_id'].count() \
        .reset_index(name='count').sort_values(by='manager_id', ascending=True)

total_points_scored_from_games['points_per_game'] = \
    total_points_scored_from_games['total_points'] / \
    total_games_from_games['count']

total_points_scored_from_games = total_points_scored_from_games.sort_values(
    by=['points_per_game', 'manager_id'],
    ascending=[False, True]
)[['manager_name', 'points_per_game']]

print(tabulate(total_points_scored_from_games, headers='keys', tablefmt='psql', showindex=False))

# Average Points Against (Regular Season)
Our unluckiest managers. Alternatively, the weakest defenses in the league 😤

In [ ]:
total_points_against_from_games = \
    regular_season_games_deduped.groupby(['manager_id', 'manager_name'])['points_against'].sum() \
        .reset_index(name='total_points_against').sort_values(by='manager_id', ascending=True)

total_games_from_games = \
    regular_season_games_deduped.groupby(['manager_id', 'manager_name'])['manager_id'].count() \
        .reset_index(name='count').sort_values(by='manager_id', ascending=True)

total_points_against_from_games['points_against_per_game'] = \
    total_points_against_from_games['total_points_against'] / \
    total_games_from_games['count']

total_points_against_from_games = total_points_against_from_games.sort_values(
    by=['points_against_per_game', 'manager_id'],
    ascending=[False, True]
)[['manager_name', 'points_against_per_game']]

print(tabulate(total_points_against_from_games, headers='keys', tablefmt='psql', showindex=False))